In [35]:
import pandas as pd
import numpy as np
import random 
from datetime import datetime, timedelta
from functools import reduce

In [3]:
# mock data
# Define columns and their datatypes based on the image
columns_dtypes = {
    # int64 columns
    "Transaction Serial No": "int64",
    "Release Number": "int64",
    "Status": "int64",
    "MTI": "int64",
    "PANNumber": "int64",
    "Processing Code": "int64",
    "Merchant Id": "int64",
    "Retrival Reference Number": "int64",
    "Account Identification1": "int64",
    "Account Identification2": "int64",
    "Confirmed": "int64",
    # float64 columns
    "Transaction Amount": "float64",
    "Card Billing Amount": "float64",
    "Expiration Date": "float64",
    "MCC": "float64",
    "POS Entry Mode": "float64",
    "POS Condition Code": "float64",
    "Auth ID Response Length": "float64",
    "Card Acceptor ID": "float64",
    "Auth ID Response": "float64",
    # datetime columns
    "Transaction Datetime": "datetime64[ns]",
    # object columns (string data)
    "Product Indicator": "object",
    "Originator Code": "object",
    "Responder Code": "object",
    "Primary Bit Map": "object",
    "Secondary Bit Map": "object",
    "Transmission Date": "object",
    "STAN": "int64",
    "Settlement Date": "object",
    "Capture Date": "object",
    "Country Code": "int64",
    "Response Code": "object",
    "Card Acceptor TerminalID": "object",
    "Terminal Owner": "object",
    "Terminal CIty": "object",
    "Terminal State": "object",
    "Terminal Country": "object",
    "Additional Data": "object",
    "Currency Code": "object",
    "Terminal Data": "object",
    "POS Data": "object",
    "Banknet Data": "object",
    "Recv Institution ID": "object",
    "Auth AgentID": "object",
    "Settlement Record": "object",
    "Batch and Shift Data": "object",
    "Settlement Data": "object",
    "Account Indicator": "object",
    "Pre authorization": "object",
    "ATM Additional Data": "object",
}


# Function to generate mock data for each column
def generate_mock_value(dtype, colname):
    if dtype == "int64":
        return np.random.randint(1000, 10000)
    elif dtype == "float64":
        return round(np.random.uniform(10.0, 1000.0), 2)
    elif dtype == "datetime64[ns]":
        return datetime.now() - timedelta(days=random.randint(0, 30))
    else:  # object
        return f"{colname[:4]}_{np.random.randint(100, 999)}"


# Create the DataFrame
num_rows = 5
data = {
    col: [generate_mock_value(dtype, col) for _ in range(num_rows)]
    for col, dtype in columns_dtypes.items()
}

df_mock = pd.DataFrame(data)

# Ensure correct dtypes
for col, dtype in columns_dtypes.items():
    if dtype.startswith("datetime"):
        df_mock[col] = pd.to_datetime(df_mock[col])
    else:
        df_mock[col] = df_mock[col].astype(dtype)

# Feature Engineering Logics

In [4]:
from src.calculation_features import generate_rolling_features, calculate_time_differences

## Time Difference Features

In [32]:
shift_columns = {
    'time_row_before': ['PANNumber'],
    'time_row_before_from_account_to_account': ['Account Identification1', 'Account Identification2'],
    'time_row_before_to_account': ['Account Identification2'],
    'time_row_before_merchant': ['PANNumber', 'Merchant Id'],
    'time_row_before_mcc': ['PANNumber', 'MCC'],
    'time_row_before_country_code': ['PANNumber', 'Country Code'],
}

time_diff_columns = {
    'TSLastTxn_mins': 'time_row_before',
    'TSLastTxn_fromacct_to_acct_mins': 'time_row_before_from_account_to_account',
    'TSLastTxn_toacct_mins': 'time_row_before_to_account',
    'TSLastTxn_merchant_mins': 'time_row_before_merchant',
    'TSLastTxn_mcc_mins': 'time_row_before_mcc',
    'TSLastTxn_countrycode_mins': 'time_row_before_country_code',
}

df_time_diff = calculate_time_differences(
    df=df_mock,
    datetime_col='Transaction Datetime',
    shift_columns=shift_columns,
    time_diff_columns=time_diff_columns
)

In [33]:
df_time_diff

,Transaction Serial No,Release Number,Status,MTI,PANNumber,Processing Code,Merchant Id,Retrival Reference Number,Account Identification1,Account Identification2,...,time_row_before_to_account,time_row_before_merchant,time_row_before_mcc,time_row_before_country_code,TSLastTxn_mins,TSLastTxn_fromacct_to_acct_mins,TSLastTxn_toacct_mins,TSLastTxn_merchant_mins,TSLastTxn_mcc_mins,TSLastTxn_countrycode_mins
4,7938,4075,4622,1051,2887,4089,6231,7656,9742,4932,...,NaT,NaT,NaT,NaT,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,5096,3326,3309,1528,8481,7989,1186,9000,4022,8204,...,NaT,NaT,NaT,NaT,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0,3454,1811,3418,3573,3621,7028,7555,3852,6961,9812,...,NaT,NaT,NaT,NaT,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,3022,1817,1644,6559,8339,6420,5410,5575,8722,1229,...,NaT,NaT,NaT,NaT,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,6487,6261,1929,4775,1066,7865,4932,3284,7450,6198,...,NaT,NaT,NaT,NaT,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


## Frequency Features

In [30]:
freq_config = [
    {
        # Transaction Count Same from Account No
        "type": "frequency",
        "groupby": "Account Identification1",
        "amount_col": "Transaction Serial No",
        "groupby_type": "No",
        "groupby_col": None,
        "windows": {
            "900S": "MFTxnCount_L15M",
            "1H": "MFTxnCount_L1H",
            "1D": "MFTxnCount_L1D",
            "7D": "MFTxnCount_L7D",
            "14D": "MFTxnCount_L14D",
            "30D": "MFTxnCount_L30D",
            "90D": "MFTxnCount_L90D",
        },
    },
    {
        # Transaction Count Same to Account No
        "type": "frequency",
        "groupby": "Account Identification1",
        "amount_col": "Transaction Serial No",
        "groupby_type": "Yes",
        "groupby_col": "Account Identification2",
        "windows": {
            "900S": "MFTxnCountSameToAcc_L15M",
            "1H": "MFTxnCountSameToAcc_L1H",
            "1D": "MFTxnCountSameToAcc_L1D",
            "7D": "MFTxnCountSameToAcc_L7D",
            "14D": "MFTxnCountSameToAcc_L14D",
            "30D": "MFTxnCountSameToAcc_L30D",
            "90D": "MFTxnCountSameToAcc_L90D",
        },
    },
]

df_freq = generate_rolling_features(
    df_mock,
    datetime_col="Transaction Datetime",
    key_col="Transaction Serial No",
    features_config=freq_config,
)

/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:43: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  .rolling(window, closed="left")
/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:43: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .rolling(window, closed="left")
/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:53: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  .rolling(window, closed="left")
/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:53: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  

In [31]:
df_freq

,Transaction Serial No,Release Number,Status,MTI,PANNumber,Processing Code,Merchant Id,Retrival Reference Number,Account Identification1,Account Identification2,...,MFTxnCount_L14D,MFTxnCount_L30D,MFTxnCount_L90D,MFTxnCountSameToAcc_L15M,MFTxnCountSameToAcc_L1H,MFTxnCountSameToAcc_L1D,MFTxnCountSameToAcc_L7D,MFTxnCountSameToAcc_L14D,MFTxnCountSameToAcc_L30D,MFTxnCountSameToAcc_L90D
0,3454,1811,3418,3573,3621,7028,7555,3852,6961,9812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6487,6261,1929,4775,1066,7865,4932,3284,7450,6198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3022,1817,1644,6559,8339,6420,5410,5575,8722,1229,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5096,3326,3309,1528,8481,7989,1186,9000,4022,8204,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7938,4075,4622,1051,2887,4089,6231,7656,9742,4932,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Monetary

In [28]:
monetary_config = [
    {
        # Average Transaction Amount
        "type": "monetary",
        "groupby": "Account Identification1",
        "amount_col": "Transaction Amount",
        "groupby_type": "No",
        "groupby_col": None,
        "windows": {
            "900S": "Avg_Amt_L15M",
            "1H": "Avg_Amt_L1H",
            "1D": "Avg_Amt_L1D",
            "7D": "Avg_Amt_L7D",
            "14D": "Avg_Amt_L14D",
            "30D": "Avg_Amt_L30D",
            "90D": "Avg_Amt_L90D",
        },
    },
    {
        # Average Transaction Amount Same to Account
        "type": "monetary",
        "groupby": "Account Identification1",
        "amount_col": "Transaction Amount",
        "groupby_type": "Yes",
        "groupby_col": "Account Identification2",
        "windows": {
            "900S": "Avg_Amt_SameToAcc_L15M",
            "1H": "Avg_Amt_SameToAcc_L1H",
            "1D": "Avg_Amt_SameToAcc_L1D",
            "7D": "Avg_Amt_SameToAcc_L7D",
            "14D": "Avg_Amt_SameToAcc_L14D",
            "30D": "Avg_Amt_SameToAcc_L30D",
            "90D": "Avg_Amt_SameToAcc_L90D",
        },
    },
    {
        # Average Card Billing Amount
        "type": "monetary",
        "groupby": "Account Identification1",
        "amount_col": "Card Billing Amount",
        "groupby_type": "No",
        "groupby_col": None,
        "windows": {
            "900S": "Avg_CardBill_Amt_L15M",
            "1H": "Avg_CardBill_Amt_L1H",
            "1D": "Avg_CardBill_Amt_L1D",
            "7D": "Avg_CardBill_Amt_L7D",
            "14D": "Avg_CardBill_Amt_L14D",
            "30D": "Avg_CardBill_Amt_L30D",
            "90D": "Avg_CardBill_Amt_L90D",
        },
    },
    {
        # Average Card Billing Amount Same to Account
        "type": "monetary",
        "groupby": "Account Identification1",
        "amount_col": "Card Billing Amount",
        "groupby_type": "Yes",
        "groupby_col": "Account Identification2",
        "windows": {
            "900S": "Avg_CardBill_Amt_SameToAcc_L15M",
            "1H": "Avg_CardBill_Amt_SameToAcc_L1H",
            "1D": "Avg_CardBill_Amt_SameToAcc_L1D",
            "7D": "Avg_CardBill_Amt_SameToAcc_L7D",
            "14D": "Avg_CardBill_Amt_SameToAcc_L14D",
            "30D": "Avg_CardBill_Amt_SameToAcc_L30D",
            "90D": "Avg_CardBill_Amt_SameToAcc_L90D",
        },
    },
    {
        # Maximum Transaction Amount
        "type": "monetary_max",
        "groupby": "Account Identification1",
        "amount_col": "Transaction Amount",
        "groupby_type": "No",
        "groupby_col": None,
        "windows": {
            "900S": "Max_Amt_L15M",
            "1H": "Max_Amt_L1H",
            "1D": "Max_Amt_L1D",
            "7D": "Max_Amt_L7D",
            "14D": "Max_Amt_L14D",
            "30D": "Max_Amt_L30D",
            "90D": "Max_Amt_L90D",
        },
    },
    {
        # Maximum Card Billing Amount
        "type": "monetary_max",
        "groupby": "Account Identification1",
        "amount_col": "Card Billing Amount",
        "groupby_type": "No",
        "groupby_col": None,
        "windows": {
            "900S": "Max_CardBill_Amt_L15M",
            "1H": "Max_CardBill_Amt_L1H",
            "1D": "Max_CardBill_Amt_L1D",
            "7D": "Max_CardBill_Amt_L7D",
            "14D": "Max_CardBill_Amt_L14D",
            "30D": "Max_CardBill_Amt_L30D",
            "90D": "Max_CardBill_Amt_L90D",
        },
    },
]

df_monetary = generate_rolling_features(
    df_mock,
    datetime_col="Transaction Datetime",
    key_col="Transaction Serial No",
    features_config=monetary_config,
)

/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:104: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  .rolling(window, closed="left")
/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:104: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .rolling(window, closed="left")
/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:114: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  .rolling(window, closed="left")
/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:114: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead

In [29]:
df_monetary

,Transaction Serial No,Release Number,Status,MTI,PANNumber,Processing Code,Merchant Id,Retrival Reference Number,Account Identification1,Account Identification2,...,Max_Amt_L14D,Max_Amt_L30D,Max_Amt_L90D,Max_CardBill_Amt_L15M,Max_CardBill_Amt_L1H,Max_CardBill_Amt_L1D,Max_CardBill_Amt_L7D,Max_CardBill_Amt_L14D,Max_CardBill_Amt_L30D,Max_CardBill_Amt_L90D
0,3454,1811,3418,3573,3621,7028,7555,3852,6961,9812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6487,6261,1929,4775,1066,7865,4932,3284,7450,6198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3022,1817,1644,6559,8339,6420,5410,5575,8722,1229,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5096,3326,3309,1528,8481,7989,1186,9000,4022,8204,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7938,4075,4622,1051,2887,4089,6231,7656,9742,4932,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Unique Count

In [ ]:
df_mock["Account Identification2 Num"], uniques = df_mock[
    "Account Identification2"
].factorize()
df_mock["Account Identification1 Num"], uniques = df_mock[
    "Account Identification1"
].factorize()

unique_count_config = [
    {
        # Unique to Account No (Same from Account No)
        "type": "unique",
        "groupby": "Account Identification1",
        "count_col": "Account Identification2 Num",
        "windows": {
            "900S": "Unique_To_Account_No_L15M",
            "1H": "Unique_To_Account_No_L1H",
            "1D": "Unique_To_Account_No_L1D",
            "7D": "Unique_To_Account_No_L7D",
            "14D": "Unique_To_Account_No_L14D",
            "30D": "Unique_To_Account_No_L30D",
            "90D": "Unique_To_Account_No_L90D",
        },
    },
    {
        # Unique from Account No (Same to Account No)
        "type": "unique",
        "groupby": "Account Identification2",
        "count_col": "Account Identification1 Num",
        "windows": {
            "900S": "Unique_From_Account_No_L15M",
            "1H": "Unique_From_Account_No_L1H",
            "1D": "Unique_From_Account_No_L1D",
            "7D": "Unique_From_Account_No_L7D",
            "14D": "Unique_From_Account_No_L14D",
            "30D": "Unique_From_Account_No_L30D",
            "90D": "Unique_From_Account_No_L90D",
        },
    },
    {
        # Unique Transaction Amount Same From Account
        "type": "unique",
        "groupby": "Account Identification1",
        "count_col": "Transaction Amount",
        "windows": {
            "900S": "Unique_AmountFrom_L15M",
            "1H": "Unique_AmountFrom_L1H",
            "1D": "Unique_AmountFrom_L1D",
            "7D": "Unique_AmountFrom_L7D",
            "14D": "Unique_AmountFrom_L14D",
            "30D": "Unique_AmountFrom_L30D",
            "90D": "Unique_AmountFrom_L90D",
        },
    },
    {
        # Unique Transaction Amount Same to Account
        "type": "unique",
        "groupby": "Account Identification2",
        "count_col": "Transaction Amount",
        "windows": {
            "900S": "Unique_AmountTo_L15M",
            "1H": "Unique_AmountTo_L1H",
            "1D": "Unique_AmountTo_L1D",
            "7D": "Unique_AmountTo_L7D",
            "14D": "Unique_AmountTo_L14D",
            "30D": "Unique_AmountTo_L30D",
            "90D": "Unique_AmountTo_L90D",
        },
    },
]

df_unique_count = generate_rolling_features(
    df_monetary,
    datetime_col="Transaction Datetime",
    key_col="Transaction Serial No",
    features_config=unique_count_config,
)

/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:161: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  .rolling(window=window, closed="left", min_periods=1)
/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:161: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .rolling(window=window, closed="left", min_periods=1)
/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:161: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  .rolling(window=window, closed="left", min_periods=1)
/home/gregorius_vidy/gbg_analytics/01 Experimentation/Model Diagnostic Suite/model-diagnostic/src/calculation_features.py:161: FutureWarning: 'H' is deprecat

In [13]:
df_unique_count

,Transaction Serial No,Release Number,Status,MTI,PANNumber,Processing Code,Merchant Id,Retrival Reference Number,Account Identification1,Account Identification2,...,Unique_AmountFrom_L14D,Unique_AmountFrom_L30D,Unique_AmountFrom_L90D,Unique_AmountTo_L15M,Unique_AmountTo_L1H,Unique_AmountTo_L1D,Unique_AmountTo_L7D,Unique_AmountTo_L14D,Unique_AmountTo_L30D,Unique_AmountTo_L90D
0,3454,1811,3418,3573,3621,7028,7555,3852,6961,9812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6487,6261,1929,4775,1066,7865,4932,3284,7450,6198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3022,1817,1644,6559,8339,6420,5410,5575,8722,1229,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5096,3326,3309,1528,8481,7989,1186,9000,4022,8204,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7938,4075,4622,1051,2887,4089,6231,7656,9742,4932,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Getting All Transformed Features

In [40]:
time_window = [
    "900S",
    "1H",
    "1D",
    "7D",
    "14D",
    "30D",
    "90D"
]

# Define the common keys for merging
merge_keys = ['Transaction Serial No', 'PANNumber']

# Helper function to extract columns based on config and time windows
def extract_columns(config, time_windows):
    return [cfg['windows'][win] for cfg in config for win in time_windows]

# Extract columns
time_diff_cols = list(time_diff_columns.keys())
freq_cols = extract_columns(freq_config, time_window)
monetary_cols = extract_columns(monetary_config, time_window)
unique_count_cols = extract_columns(unique_count_config, time_window)

# Subset dataframes
df_time_diff = df_time_diff[merge_keys + time_diff_cols]
df_freq = df_freq[merge_keys + freq_cols]
df_monetary = df_monetary[merge_keys + monetary_cols]
df_unique_count = df_unique_count[merge_keys + unique_count_cols]

# Merge all feature dataframes + additional features
dfs_to_merge = [df_time_diff, df_freq, df_monetary, df_unique_count]
df_final = reduce(lambda left, right: pd.merge(left, right, on=merge_keys, how='outer'), dfs_to_merge)
df_final = df_final.merge(
    df_mock[merge_keys + ['Transaction Amount', 'Card Billing Amount', 'Confirmed']],
    on=merge_keys,
    how='left'
)

In [41]:
df_final

,Transaction Serial No,PANNumber,TSLastTxn_mins,TSLastTxn_fromacct_to_acct_mins,TSLastTxn_toacct_mins,TSLastTxn_merchant_mins,TSLastTxn_mcc_mins,TSLastTxn_countrycode_mins,MFTxnCount_L15M,MFTxnCount_L1H,...,Unique_AmountTo_L15M,Unique_AmountTo_L1H,Unique_AmountTo_L1D,Unique_AmountTo_L7D,Unique_AmountTo_L14D,Unique_AmountTo_L30D,Unique_AmountTo_L90D,Transaction Amount,Card Billing Amount,Confirmed
0,3022,8339,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,746.66,209.50,8010
1,3454,3621,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,623.59,139.58,7640
2,5096,8481,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,955.41,674.45,5487
3,6487,1066,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,574.86,480.66,8978
4,7938,2887,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,868.09,380.46,6630
